In [1]:
useMock=False

In [2]:
from cffi import FFI
ffi = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
ffi.cdef(myHeaders)
with open('extracted_headers/safe_c_ffi_funcs.h','r') as f:
    funs=f.read()
ffi.cdef(funs)
lib=ffi.dlopen('../compiled_binaries/libsafe_app' + useMock*'_mock' + '.so')

---

maybe it makes most sense to really define the data types by hand and just check them for changes with updates on the api - not entirely sure

- rid

---

```c
typedef unsigned long int uintptr_t;

typedef struct {
 char* id;
 char* scope;
 char* name;
 char* vendor;
} AppExchangeInfo;
```

In [3]:
class AppExchangeInfo:
    def __init__(self,id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
        self.id = ffi.new('char[]',id)
        self.scope = ffi.new('char[]',scope)
        self.name = ffi.new('char[]',name)
        self.vendor = ffi.new('char[]',vendor)

        self.entity = ffi.new('AppExchangeInfo *',[self.id,self.scope,self.name,self.vendor])

In [4]:
def AppExchangeInfo(id=b'noId',scope=b'noScope',name=b'noName',vendor=b'nobody'):
    id = ffi.new('char[]',id)
    scope = ffi.new('char[]',scope)
    name = ffi.new('char[]',name)
    vendor = ffi.new('char[]',vendor)

    myStruct = ffi.new('AppExchangeInfo *',[id,scope,name,vendor])
    
    return myStruct, [id, scope, name, vendor]

newExChangeInfo,infopayload = AppExchangeInfo(b'0.0.2',name=b'pyTest',vendor=b'rid')

newExChangeInfo,infopayload = AppExchangeInfo(id=b'meh',scope=b'myScope',
                                              name=b'PyTests only today',vendor=b'rid')

ffi.string(newExChangeInfo.vendor)

ffi.string(newExChangeInfo.name)

```c
typedef struct {
 _Bool read;
 _Bool insert;
 _Bool update;
 _Bool delete;
 _Bool manage_permissions;
} PermissionSet;
```

In [5]:
def PermissionSet(read=True,insert=True,update=True,delete=True,manage_permissions=True):
    return ffi.new('PermissionSet *',[read,insert,update,delete,manage_permissions])

newPermissionSet = PermissionSet()

```c
typedef struct {
 char* cont_name;
 PermissionSet access;
} ContainerPermissions;
```

In [6]:
def ContainerPermissions(name=b'noName',access=None):
    containerName = ffi.new('char[]',name)
    if not access:
        access = PermissionSet()
    container = ffi.new('ContainerPermissions *',[containerName,access[0]])
    
    return container, [containerName,access]

newPermissions,permission_addon=ContainerPermissions(b'_pictures',newPermissionSet)

In [7]:
newPermissions,permission_addon=ContainerPermissions(b'_publicNames')

In [8]:
newPermissions,permission_addon=ContainerPermissions(b'_music')

In [9]:
ffi.string(newPermissions.cont_name)

b'_music'

```c
typedef struct {
 AppExchangeInfo app;
 _Bool app_container;
 ContainerPermissions* containers;
 uintptr_t containers_len;
 uintptr_t containers_cap;
} AuthReq;
```

In [10]:
def AuthReq(permissions,containers_len,containers_cap,id=b'noId',scope=b'pythonscript',
            name=b'noName',vendor=b'nobody',app_container=True):
    
    newExChangeInfo,infopayload = AppExchangeInfo(id,scope,name,vendor)
    
    authReq = ffi.new('AuthReq *',[newExChangeInfo[0],app_container,permissions,containers_len,containers_cap])
    
    return authReq, [newExChangeInfo,infopayload]

myAuth,addData=AuthReq(newPermissions,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'rid never work',app_container=True)

myAuth,addData=AuthReq(newPermissions,0,0,id=b'ridsTest',scope=b'pythonscript'
                       ,name=b'PyTest',vendor=b'nobody',app_container=True)

In [11]:
myAuth,addData=AuthReq(newPermissions,1,1,id=b'newProgId_unique_thing',scope=b'lalali'
                       ,name=b'newProgramName',vendor=b'nobody',app_container=True)

In [12]:
@ffi.callback("void(void *, FfiResult *)")
def feedback(userData, result):
    print(userData)
    print(result.error_code)

In [13]:
lib.app_set_additional_search_path(ffi.new('char[]',b'../compiled_binaries/'),ffi.NULL,feedback)

<cdata 'void *' NULL>
0


@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, noClueWhatNumber, resultString):
    print(userData)
    print(result)
    print(noClueWhatNumber)
    print(resultString)

lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

myAuth.containers.access.insert=True

myAuth.containers.access.delete=True

myAuth.containers.access.read=True

In [14]:
ffi.string(myAuth.app.name)

b'newProgramName'

In [15]:
ffi.string(myAuth.app.id)

b'newProgId_unique_thing'

In [16]:
encodedAuth=None

In [17]:
@ffi.callback("void(void *, FfiResult *, unsigned long int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [18]:
@ffi.callback("void(void *, FfiResult *, unsigned int, char *)")
def o_cb(userData, result, reqId, encodedReq):
    global encodedAuth
    print(userData)
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(reqId)
    print(ffi.string(encodedReq))
    encodedAuth = ffi.string(encodedReq)

In [19]:
lib.encode_auth_req(myAuth,ffi.NULL,o_cb)

<cdata 'void *' NULL>
0
4042140874
b'AAAAAMos7vAAAAAAFgAAAAAAAABuZXdQcm9nSWRfdW5pcXVlX3RoaW5nAQYAAAAAAAAAbGFsYWxpDgAAAAAAAABuZXdQcm9ncmFtTmFtZQYAAAAAAAAAbm9ib2R5AQEAAAAAAAAABgAAAAAAAABfbXVzaWMFAAAAAAAAAAAAAAABAAAAAgAAAAMAAAAEAAAA'


In [20]:
b'safe-auth://' + encodedAuth

b'safe-auth://AAAAAMos7vAAAAAAFgAAAAAAAABuZXdQcm9nSWRfdW5pcXVlX3RoaW5nAQYAAAAAAAAAbGFsYWxpDgAAAAAAAABuZXdQcm9ncmFtTmFtZQYAAAAAAAAAbm9ib2R5AQEAAAAAAAAABgAAAAAAAABfbXVzaWMFAAAAAAAAAAAAAAABAAAAAgAAAAMAAAAEAAAA'

In [21]:
import base64

In [22]:
schemeName=b'safe-'+base64.encodebytes(ffi.string(myAuth.app.id))[:-2]

In [23]:

sysUri = FFI()
with open('extracted_headers/safe_c_ffi_data.h','r') as f:
    myHeaders=f.read()
sysUri.cdef(myHeaders)
with open('extracted_headers/system_uri.h','r') as f:
    funs=f.read()
sysUri.cdef(funs)
lib2=sysUri.dlopen('../compiled_binaries/libsystem_uri.so')

In [24]:
@sysUri.callback("void(void *, FfiResult *)")
def installerFeedback(userData, result):
    print(userData)
    print(result.error_code)

```c
void install(char* bundle, 
             char* vendor, 
             char* name, 
             char* exec_args, 
             uint_least64_t exec_args_len, 
             char* icon, 
             char* schemes, 
             void* user_data, 
             void (*o_cb)(void* user_data, FfiResult* result));
```

In [25]:
print(ffi.string(myAuth.app.name))
print(ffi.string(myAuth.app.vendor))
print(ffi.string(myAuth.app.scope))

b'newProgramName'
b'nobody'
b'lalali'


In [26]:
import os
print(os.getcwd())

/home/yomuga/maidsafe/pySafe/pySafe


In [27]:
pathToHandler=(os.getcwd()+'/uriHandler.py').encode()

In [28]:
pathToHandler

b'/home/yomuga/maidsafe/pySafe/pySafe/uriHandler.py'

```c
myAuth,addData=AuthReq(newPermissions,1,1,id=b'newProgId',scope=b'lalali'
                       ,name=b'newProgramName',vendor=b'nobody',app_container=False)
```

In [29]:
bundle=sysUri.new('char[]',ffi.string(myAuth.app.id))
vendor=sysUri.new('char[]',ffi.string(myAuth.app.vendor))
name=sysUri.new('char[]',ffi.string(myAuth.app.name))
exec_args=sysUri.new('char[]',b'python')
exec_args2=sysUri.new('char[]',pathToHandler)
exec_args_len=2
icon=sysUri.new('char[]',b'')
schemes=sysUri.new('char[]',schemeName)
user_data=sysUri.NULL

In [30]:
exec_args_1 = sysUri.new('char*[]',[exec_args,exec_args2])

In [31]:
lib2.install(bundle,vendor,name,exec_args_1 ,exec_args_len,icon,schemes,user_data,installerFeedback)

<cdata 'void *' NULL>
0


```c 
void open_uri(char* uri, 
              void* user_data, 
              void (*o_cb)(void* user_data, FfiResult* result));
```

In [32]:
@sysUri.callback("void(void *, FfiResult *)")
def openerFeedback(userData, result):
    global res
    res = result
    print(userData)
    print(result.error_code)

In [33]:
ffi.string(myAuth.app.name)

b'newProgramName'

In [34]:
ffi.string(myAuth.app.vendor)

b'nobody'

In [35]:
ffi.string(myAuth.app.name)

b'newProgramName'

In [36]:
uri=sysUri.new('char[]',b'safe-auth://' + encodedAuth)
user_data=sysUri.NULL

In [37]:
lib2.open_uri(uri,user_data,openerFeedback)

<cdata 'void *' NULL>
0


In [38]:
def printCrustConfig(configAddress,configlength=42):
    lali=[]
    for i in range(configlength):
        lali.append(configAddress[i])
    print(lali)

```c 
void decode_ipc_msg(char* msg, 
                    void* user_data, 
                    void (*o_auth)(void* user_data, uint32_t req_id, AuthGranted* auth_granted), 
                    void (*o_unregistered)(void* user_data, uint32_t req_id, uint8_t* serialised_cfg, 
                                            uintptr_t serialised_cfg_len), 
                    void (*o_containers)(void* user_data, uint32_t req_id), 
                    void (*o_share_mdata)(void* user_data, uint32_t req_id), 
                    void (*o_revoked)(void* user_data), 
                    void (*o_err)(void* user_data, FfiResult* result, uint32_t req_id));
```

In [39]:
import time

In [40]:
import safeUtils

In [41]:
@ffi.callback("void(void *, uint32_t, AuthGranted *)")
def o_auth_decode_ipc_msg(userData, req_id, auth_granted):
    
    printCrustConfig(auth_granted.bootstrap_config)
    
    print('o_auth')
    print(userData)
    print(req_id)
    returnDict = ffi.from_handle(userData)
    #returnDict['AuthGranted'] = picData
    returnDict['AuthGranted'] = safeUtils.copy(auth_granted,ffi)
    #print(returnDict)
    myContainer=auth_granted.access_container_entry.containers[0]
    print(myContainer)
    print(auth_granted.access_container_entry.containers_len)
    print(auth_granted.access_container_entry.containers_cap)
    print(ffi.string(myContainer.name))
    print(myContainer.mdata_info)
    print(ffi.sizeof(myContainer.mdata_info))
    print(myContainer.permissions)
    print(ffi.sizeof(myContainer.permissions))


In [42]:
@ffi.callback("void(void *, uint32_t, uint8_t *, uintptr_t)")
def o_unregistered_decode_ipc_msg(userData, req_id, serialised_cfg, serialised_cfg_len):
    print('o_unregistered')
    print(req_id)

In [43]:
@ffi.callback("void(void *, uint32_t)")
def o_containers_decode_ipc_msg(userData, req_id):
    print('o_containers')
    print(req_id)

In [44]:
@ffi.callback("void(void *, uint32_t)")
def o_share_mdata_decode_ipc_msg(userData, req_id):
    print('o_share')
    print(req_id)

In [45]:
@ffi.callback("void(void *)")
def o_revoked_decode_ipc_msg(userData):
    print('o_revoked')
    print(req_id)

In [46]:
@ffi.callback("void(void *, FfiResult *, uint32_t)")
def o_error_decode_ipc_msg(userData, result, req_id):
    global res
    res = result
    print('o_err')
    print(userData)
    print(req_id)
    print(result.error_code)
    print(ffi.string(result.description))

In [47]:
with open('answer','r') as f:
    answer=f.readlines()

In [48]:
import re
data=re.compile("'safe-.+?:(.+?)'")

In [49]:
my=data.findall(answer[-1])

In [50]:
msg = ffi.new('char[]',my[-1].encode())

In [51]:
my[-1]

'AQAAAJPycNYAAAAAAAAAACAAAAAAAAAAbhJf-9V6xLIwtiYS1eDRWmvSSRSBSUm2bmaPvH9TQoYgAAAAAAAAAIo_Z5jHhCW8qHdeEXdC-u9egZbqFSwbyVwzFouX10YhIAAAAAAAAAC3wuVj6VhDAa1wCgFrh0qrYKjV2N-LT7jO0a6mVftF4UAAAAAAAAAACjUppInzwzZVU4PMR2hVF5ynLzGomtRjvsmheJtPEYa3wuVj6VhDAa1wCgFrh0qrYKjV2N-LT7jO0a6mVftF4SAAAAAAAAAAZBZgo04W7bAvKldm2rzn3DTsmEFlp5e2KfFnVgR9Y1cgAAAAAAAAAH-63-1sn0cNPuZcpqRNRUQefQepWo-Ykx8jWP1IDl6qGQAAAAAAAAAQAAAAAAAAADE3OC42Mi43Ni44OjU0ODMTAAAAAAAAADEzOC42OC4xODUuMjE4OjU0ODMSAAAAAAAAADEzOC42OC4xODEuNTc6NTQ4MxIAAAAAAAAAMTM4LjY4LjE4MS42MDo1NDgzEgAAAAAAAAAxMzguNjguMTgxLjg2OjU0ODMSAAAAAAAAADEzOC42OC4xODEuODc6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xNjg6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xNzY6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xNzk6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xODA6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4xODI6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4yNDI6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4yNDM6NTQ4MxMAAAAAAAAAMTM4LjY4LjE4MS4yNDk6NTQ4MxIAAAAAAAAAMTM4LjY4LjE4OS4xNDo1NDgzEgAAAAAAAAAxMzguNjguMTg5LjE1OjU0ODMSAAAAAAAAADEzOC42OC4xODkuMTc6NTQ4MxIAAAA

In [52]:
returnDict={}

In [53]:
myHand=ffi.new_handle(returnDict)

In [54]:
lib.decode_ipc_msg(msg,myHand,o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

[25, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 49, 55, 56, 46, 54, 50, 46, 55, 54, 46, 56, 58, 53, 52, 56, 51, 19, 0, 0, 0, 0, 0, 0, 0, 49, 51]
o_auth
<cdata 'void *' 0x7f3aec0cf6d8>
3597726355
<cdata 'ContainerInfo &' 0x56517636bac0>
1
1
b'apps/okay this is my program id'
<cdata 'MDataInfo &' 0x56517636bac8>
160
<cdata 'PermissionSet &' 0x56517636bb68>
5


#time.sleep(3)
lib.decode_ipc_msg(msg,ffi.new_handle(returnDict),o_auth_decode_ipc_msg,o_unregistered_decode_ipc_msg
                   ,o_containers_decode_ipc_msg,o_share_mdata_decode_ipc_msg,
                   o_revoked_decode_ipc_msg,o_error_decode_ipc_msg)

In [55]:
granted=returnDict['AuthGranted'][0]

In [56]:
conTest=ffi.new('ContainerInfo *')

In [57]:
ffi.sizeof(conTest[0])

176

In [58]:
ffi.sizeof(conTest.mdata_info)

160

In [59]:
ffi.sizeof(conTest.permissions)

5

In [60]:
ffi.sizeof(conTest.name)

8

In [61]:
160+5+8

173

----

now the app

ffi.cdef('''
typedef struct { ...; } foo_t;
''')

In [62]:
@ffi.callback("void(void*)")
def disconnect_app_registered(user_data):
    print('disconnected - callback from app_registered')

In [63]:
innerData=None
@ffi.callback("void(void* , FfiResult* , App* app)")
def result_app_registered(user_data, result, app):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)
    if app != ffi.NULL:
        returnDict['app'] = safeUtils.copy(app,ffi)
    print(returnDict['app'][0][0])
    print(app[0])
    returnDict['test']=ffi.new_handle(app[0])
    print(ffi.buffer(app[0],8)[:])
    print(ffi.buffer(returnDict['app'][0],8)[:])
    #print(ffi.from_buffer(app[0][0]))
    print('now the interesting part')
    #makadata = ffi.new()
    print(app)
    returnDict['test3'] = ffi.new('foo_t *',app[0][0])

lib.app_registered(myAuth.app.id,grantedAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

In [64]:
returnDict['AuthGranted'][0]

<cdata 'AuthGranted *' owning 336 bytes>

In [65]:
lib.app_registered(myAuth.app.id,returnDict['AuthGranted'][0],ffi.new_handle(returnDict),disconnect_app_registered,result_app_registered)

results
error code: 
0
<cdata 'void *' 0x565175fbcf50>
<cdata 'void *' 0x565175fbcf50>
b'\x00\x00\x00\x00\x00\x00\x00\x00'
b'P\xcf\xfbuQV\x00\x00'
now the interesting part
<cdata 'void * *' 0x565176536110>


From cffi callback <function result_app_registered at 0x7f3ade706bf8>:
Traceback (most recent call last):
  File "<ipython-input-63-1e6ddec253e1>", line 21, in result_app_registered
TypeError: cdata 'void' is opaque


In [79]:
returnDict['test3'][0]

<cdata 'void *' 0x55aacba31c10>

In [80]:
returnDict['app'][0][0]

<cdata 'void *' 0x55aacba31c10>

In [65]:
ffi.new('App *')

<cdata 'void *' NULL>

In [66]:
ffi.new_handle(returnDict)

<cdata 'void *' handle to {'AuthGranted': (<cdata 'AuthGranted *' owning 336 bytes>, [<cdata 'unsigned char[]' owning 689 bytes>, <cdata 'ContainerInfo[]' owning 176 bytes>, [<cdata 'char[]' owning 32 bytes>]]), 'app': (<cdata 'void * *' owning 8 bytes>, []), 'test': <cdata 'void *' handle to <cdata 'void *' 0x5636aa137f90>>}>

In [71]:
returnDict

{'AuthGranted': (<cdata 'AuthGranted *' owning 336 bytes>,
  [<cdata 'unsigned char[]' owning 689 bytes>,
   <cdata 'ContainerInfo[]' owning 176 bytes>,
   [<cdata 'char[]' owning 32 bytes>]]),
 'app': (<cdata 'struct $$App * *' owning 8 bytes>, []),
 'mdata': 0,
 'test': <cdata 'void *' handle to <cdata 'struct $$App *' 0x561c8267ef70>>}

In [84]:
returnDict['app'][0][0]

<cdata 'void *' 0x5623ec933280>

In [67]:
print(ffi.buffer(returnDict['app'][0],8)[:])

b'0\x82\xf2Z2V\x00\x00'


In [68]:
with open('picData.jpeg','rb') as f:
    picData = f.read()

In [67]:
@ffi.callback("void(void* , FfiResult* , unsigned long)")
def result_mdata_entries_new(user_data, result, entriesHandle):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)
    returnDict['mdata'] = entriesHandle

In [74]:
lib.mdata_entries_new(returnDict['app'][0],ffi.new_handle(returnDict),result_mdata_entries_new)

results
error code: 
-2000
b'Unexpected (probably a logic error): panic'


In [72]:
lib.mdata_entries_new(returnDict['app'][0],ffi.new_handle(returnDict),result_mdata_entries_new)

results
error code: 
-2000
b'Unexpected (probably a logic error): panic'


In [70]:
lib.mdata_entries_new(returnDict['test3'],ffi.new_handle(returnDict),result_mdata_entries_new)

results
error code: 
-2000
b'Unexpected (probably a logic error): panic'


In [90]:
returnDict

{'AuthGranted': (<cdata 'AuthGranted *' owning 336 bytes>,
  [<cdata 'unsigned char[]' owning 689 bytes>,
   <cdata 'ContainerInfo[]' owning 176 bytes>,
   [<cdata 'char[]' owning 32 bytes>]]),
 'app': (<cdata 'void * *' owning 8 bytes>, []),
 'mdata': 0,
 'test': <cdata 'void *' handle to <cdata 'void *' 0x55af8f6d4440>>}

In [66]:
@ffi.callback("void(void* , FfiResult* , AccountInfo*)")
def result_app_account_info(user_data, result, info):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)
    if info != ffi.NULL:
        returnDict['info'] = safeUtils.copy(info,ffi)

In [67]:
lib.app_account_info(returnDict['app'][0],ffi.new_handle(returnDict),result_app_account_info)

results
error code: 
-2000
b'Unexpected (probably a logic error): panic'


In [64]:
@ffi.callback("void(void *, FfiResult *, ContainerPermissions *, unsigned long)")
def result_access_container_fetch(user_data, result, permissions, permissionLength):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)
    if permissions != ffi.NULL:
        returnDict['info'] = safeUtils.copy(permissions[0],ffi)

In [65]:
lib.access_container_fetch(returnDict['app'][0],ffi.new_handle(returnDict),result_access_container_fetch)

results
error code: 
-2000
b'Unexpected (probably a logic error): panic'


In [125]:
@ffi.callback("void(void *, FfiResult *)")
def result_app_init_logging(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    returnDict = ffi.from_handle(user_data)

In [126]:
output_file_name=ffi.new('char[]',b'/home/yomuga/maidsafe/pySafe/pySafe/log_out')

In [127]:
lib.app_init_logging(output_file_name,ffi.new_handle(returnDict),result_app_init_logging)

results
error code: 
-2000
b'Unexpected (probably a logic error): Logger already initialised'


In [128]:
@ffi.callback("void(void *, FfiResult *, char *)")
def result_app_output_log_path(user_data, result,logFile):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))
    print(ffi.string(logFile))
    returnDict['filePath'] = ffi.string(logFile)

In [131]:
file_name=ffi.new('char[]',b'log_out2')

In [132]:
lib.app_output_log_path(file_name,ffi.new_handle(returnDict),result_app_output_log_path)

results
error code: 
0
b'../compiled_binaries/log_out2'


In [68]:
newMdata=ffi.new('MDataInfo *')

In [69]:
newMdata.type_tag=15000

In [70]:
for i in range(32):
    newMdata.name[i]=i+1

In [71]:
@ffi.callback("void(void *, FfiResult *)")
def result_mdata_put(user_data, result):
    print('results')
    print('error code: ')
    print(result.error_code)
    if result.error_code != 0:
        print(ffi.string(result.description))

In [72]:
lib.mdata_put(returnDict['app'][0],newMdata,0,0,ffi.new_handle(returnDict),result_mdata_put)

results
error code: 
-2000
b'Unexpected (probably a logic error): panic'


In [147]:
returnDict['app'][0]

<cdata 'void * *' owning 8 bytes>

In [97]:
returnDict

{'AuthGranted': (<cdata 'AuthGranted *' owning 336 bytes>,
  [<cdata 'unsigned char[]' owning 689 bytes>,
   <cdata 'ContainerInfo[]' owning 176 bytes>,
   [<cdata 'char[]' owning 32 bytes>]]),
 'app': (<cdata 'void * *' owning 8 bytes>, [])}

In [87]:
returnDict['app'][0]

<cdata 'void * *' owning 8 bytes>

In [81]:
returnDict

{'AuthGranted': (<cdata 'AuthGranted *' owning 336 bytes>,
  [<cdata 'unsigned char[]' owning 689 bytes>,
   <cdata 'ContainerInfo[]' owning 176 bytes>,
   [<cdata 'char[]' owning 32 bytes>]])}

In [85]:
ffi.sizeof(ffi.new_handle(returnDict))

8

In [88]:
ffi.sizeof(picDataHandle)

8

In [92]:
picDataHandle

<cdata 'void *' handle to b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00C\x00\x02\x02\x02\x02\x02\x01\x02\x02\x02\x02\x03\x02\x02\x03\x03\x06\x04\x03\x03\x03\x03\x07\x05\x05\x04\x06\x08\x07\t\x08\x08\x07\x08\x08\t\n\r\x0b\t\n\x0c\n\x08\x08\x0b\x0f\x0b\x0c\r\x0e\x0e\x0f\x0e\t\x0b\x10\x11\x10\x0e\x11\r\x0e\x0e\x0e\xff\xdb\x00C\x01\x02\x03\x03\x03\x03\x03\x07\x04\x04\x07\x0e\t\x08\t\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\xff\xc0\x00\x11\x08\x02\x80\x02\x80\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1e\x00\x00\x00\x06\x03\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x03\x04\x05\x06\x07\x00\x01\x08\t\n\xff\xc4\x00J\x10\x00\x01\x03\x03\x03\x02\x04\x04\x04\x04\x04\x04\x03\x07\x02\x07\x01\x02\x03\x04\x00\x05\x11\x06\x12!1A\x07\x13"Q\x142aq\x08#B\x81\x153R\x91$Cb\xa1\x16r\xb1\xc1%S\xd1\t\x174c

In [77]:
returnDict

{'AuthGranted': (<cdata 'AuthGranted *' owning 336 bytes>,
  [<cdata 'unsigned char[]' owning 689 bytes>,
   <cdata 'ContainerInfo[]' owning 176 bytes>,
   [<cdata 'char[]' owning 32 bytes>]]),
 'app': (<cdata 'void * *' owning 8 bytes>, [])}

In [78]:
myApp=returnDict['app']

In [ ]:
myApp=ffi.from_handle(myApp[0][0])

In [ ]:
lib.app_account_info()

In [86]:
with open('picData.jpeg','rb') as f:
    picData = f.read()

In [71]:
with open('picOut.jpeg','wb') as f:
    f.write(picData)

In [87]:
picDataHandle=ffi.new_handle(picData)

In [73]:
grantedAuth

<cdata 'AuthGranted *' owning 336 bytes>

In [63]:
grantedAuth.bootstrap_config = bootstrap_config

In [64]:
grantedAuth.bootstrap_config

<cdata 'unsigned char *' 0x55b1e87cdb20>

In [65]:
lib.app_registered(myAuth.app.id,grantedAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

results
error code: 
0


In [65]:
myApp[0]

<cdata 'void *' 0x55acbc6d6c20>

In [163]:
ffi.sizeof(grantedAuth.access_container_entry.containers)

8

In [166]:
ffi.sizeof(grantedAuth.bootstrap_config)

8

In [170]:
grantedAuth.bootstrap_config_len

16

In [182]:
test=ffi.from_buffer(ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:])

In [205]:
getCopy(grantedAuth)

(<cdata 'AuthGranted *' owning 336 bytes>,
 <cdata 'char[]' buffer len 16 from 'bytes' object>)

In [204]:
def getCopy(data):
    if ffi.typeof(grantedAuth) == ffi.typeof(ffi.new('AuthGranted *')):
        newData = ffi.new('AuthGranted *',data[0])
        bootstrapData = ffi.new('unsigned char[]',ffi.buffer(data.bootstrap_config,int(data.bootstrap_config_len))[:])
        newData.bootstrap_config = bootstrapData
        return newData,bootstrapData

In [282]:
test=ffi.new('short int[]',[1,4])

In [285]:
test=ffi.new('short int *',1)

In [284]:
test

<cdata 'short[]' owning 4 bytes>

In [272]:
ffi.sizeof(test)

8

In [286]:
ffi.buffer(test,ffi.sizeof(test))[:]=ffi.buffer(test,ffi.sizeof(test))[:]

b'\x01\x00\x00\x00\x00\x00\x00\x00'

In [274]:
ffi.alignof(ffi.typeof(test))

8

In [308]:
ffi.new(ffi.typeof(grantedAuth),grantedAuth[0])

<cdata 'AuthGranted *' owning 336 bytes>

In [295]:
grantedAuth.access_container_entry.containers

<cdata 'ContainerInfo *' 0x8>

In [295]:
grantedAuth.access_container_info.

<cdata 'ContainerInfo *' 0x8>

In [295]:
grantedAuth.access_container_entry.containers[0]

<cdata 'ContainerInfo *' 0x8>

In [300]:
ffi.string(ffi.new('char[]',b'haha'))

b'haha'

In [68]:
def getCopy(data):
    if ffi.typeof(data) == ffi.typeof(ffi.new('FfiResult *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        if data.description != ffi.NULL:
            description = ffi.new('char[]',ffi.string(data.description))
        else:
            description = ffi.NULL
        newData.description = description
        
        return newData,[description]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('File *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        user_metadata_ptr = ffi.new('uint8_t[]',data.user_metadata_len)
        copyLen=int(ffi.sizeof(data.user_metadata_ptr)*data.user_metadata_len)
        ffi.buffer(user_metadata_ptr,copyLen)[:]=ffi.buffer(data.user_metadata_ptr,copyLen)[:]
        newData.user_metadata_ptr = user_metadata_ptr
        
        return newData,[user_metadata_ptr]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('AppExchangeInfo *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        id = ffi.new('char[]',ffi.string(data.id))
        scope = ffi.new('char[]',ffi.string(data.scope))
        name = ffi.new('char[]',ffi.string(data.name))
        vendor = ffi.new('char[]',ffi.string(data.vendor))
        newData.id = id
        newData.scope = scope
        newData.name = name
        newData.vendor = vendor
        
        return newData,[id, scope, name, vendor]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('ContainerPermissions *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        cont_name = ffi.new('char[]',ffi.string(data.cont_name))
        newData.cont_name = cont_name
        
        return newData,[cont_name]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('AuthReq *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        containers = ffi.new('ContainerPermissions[]',data.containers_len)
        copyLen=int(ffi.sizeof(data.containers)*data.containers_len)
        ffi.buffer(containers,copyLen)[:]=ffi.buffer(data.containers,copyLen)[:]
        newData.containers = containers
        containerNames=[]
        for i in range(data.containers_len):
            containerNames.append(ffi.new('char[]',ffi.string(data.containers[i].cont_name)))
            newData.containers[i].cont_name=containerNames[-1]
        
        return newData,[containers,containerNames]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('ContainersReq *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        containers = ffi.new('ContainerPermissions[]',data.containers_len)
        copyLen=int(ffi.sizeof(data.containers)*data.containers_len)
        ffi.buffer(containers,copyLen)[:]=ffi.buffer(data.containers,copyLen)[:]
        newData.containers = containers
        containerNames=[]
        for i in range(data.containers_len):
            containerNames.append(ffi.new('char[]',ffi.string(data.containers[i].cont_name)))
            newData.containers[i].cont_name=containerNames[-1]
        
        return newData,[containers,containerNames]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('ShareMDataReq *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        mdata = ffi.new('ShareMData[]',data.mdata_len)
        copyLen=int(ffi.sizeof(data.mdata)*data.mdata_len)
        ffi.buffer(mdata,copyLen)[:]=ffi.buffer(data.mdata,copyLen)[:]
        newData.mdata = mdata
        
        return newData,[mdata]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('ContainerInfo *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        name = ffi.new('char[]',ffi.string(data.name))
        newData.name = name
        
        return newData,[name]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('AccessContainerEntry *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        containers = ffi.new('ContainerInfo[]',data.containers_len)
        copyLen=int(ffi.sizeof(data.containers)*data.containers_len)
        ffi.buffer(containers,copyLen)[:]=ffi.buffer(data.containers,copyLen)[:]
        newData.containers = containers
        
        return newData,[containers]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('AuthGranted *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        bootstrap_config = ffi.new('unsigned char[]',data.bootstrap_config_len)
        copyLen=int(ffi.sizeof(data.bootstrap_config)*data.bootstrap_config_len)
        ffi.buffer(bootstrap_config,copyLen)[:]=ffi.buffer(data.bootstrap_config,copyLen)[:]
        newData.bootstrap_config = bootstrap_config
        
        containers = ffi.new('ContainerInfo[]',data.access_container_entry.containers_len)
        copyLen=int(ffi.sizeof(data.access_container_entry.containers)*data.access_container_entry.containers_len)
        ffi.buffer(containers,copyLen)[:]=ffi.buffer(data.access_container_entry.containers,copyLen)[:]
        newData.access_container_entry.containers = containers
        containerNames=[]
        for i in range(data.access_container_entry.containers_len):
            containerNames.append(ffi.new('char[]',ffi.string(data.access_container_entry.containers[i].name)))
            newData.access_container_entry.containers[i].name=containerNames[-1]
        
        return newData,[bootstrap_config,containers,containerNames]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('AppAccess *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        name = ffi.new('char[]',ffi.string(data.name))
        app_id = ffi.new('char[]',ffi.string(data.app_id))
        newData.name = name
        newData.app_id = app_id
        
        return newData,[name, app_id]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('MetadataResponse *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        name = ffi.new('char[]',ffi.string(data.name))
        description = ffi.new('char[]',ffi.string(data.description))
        newData.name = name
        newData.description = description
        
        return newData,[name, description]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('MDataKey *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        val = ffi.new('uint8_t[]',data.val_len)
        copyLen=int(ffi.sizeof(data.val)*data.val_len)
        ffi.buffer(val,copyLen)[:]=ffi.buffer(data.val,copyLen)[:]
        newData.val = val
        
        return newData,[val]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('MDataValue *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        content = ffi.new('uint8_t[]',data.content_len)
        copyLen=int(ffi.sizeof(data.content)*data.content_len)
        ffi.buffer(content,copyLen)[:]=ffi.buffer(data.content,copyLen)[:]
        newData.content = content
        
        return newData,[content]
    
    elif ffi.typeof(data) == ffi.typeof(ffi.new('MDataEntry *')):
        newData = ffi.new(ffi.typeof(data),data[0])
        
        val = ffi.new('uint8_t[]',data.key.val_len)
        copyLen=int(ffi.sizeof(data.key.val)*data.key.val_len)
        ffi.buffer(val,copyLen)[:]=ffi.buffer(data.key.val,copyLen)[:]
        newData.key.val = val
        
        content = ffi.new('uint8_t[]',data.value.content_len)
        copyLen=int(ffi.sizeof(data.value.content)*data.value.content_len)
        ffi.buffer(content,copyLen)[:]=ffi.buffer(data.value.content,copyLen)[:]
        newData.value.content = content
        
        return newData,[val,content]
    
    else:
        newData = ffi.new(ffi.typeof(data),data[0])
        return newData,[]

In [66]:
import safeUtils

In [ ]:
safeUtils.

In [68]:
import importlib

In [72]:
importlib.reload(safeUtils)

ModuleNotFoundError: No module named 'pySafe'

In [70]:
safeUtils.copy(grantedAuth)

CDefError: cannot parse "FfiResult *"
<cdef source string>:2:11: before: *

In [69]:
getCopy(grantedAuth)

(<cdata 'AuthGranted *' owning 336 bytes>,
 [<cdata 'unsigned char[]' owning 16 bytes>,
  <cdata 'ContainerInfo[]' owning 0 bytes>,
  []])

In [208]:
ffi.unpack('int[]',ffi.from_buffer(ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:]))

TypeError: unpack() argument 1 must be _cffi_backend.CData, not str

In [208]:
ffi.unpack('int[]',ffi.from_buffer(ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:]))

TypeError: unpack() argument 1 must be _cffi_backend.CData, not str

In [193]:
ffi.typeof(grantedAuth) == ffi.typeof(ffi.new('AuthGranted *'))

True

In [194]:
ffi.typeof(grantedAuth.app_keys) == ffi.typeof(ffi.new('AuthGranted *'))

False

'AuthGranted *'

In [184]:
ffi.cast('int[]',test)

TypeError: cannot cast to ctype 'int[]'

In [199]:
grantedAuth.bootstrap_config[0]

0

In [201]:
ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [195]:
ffi.from_buffer(ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:])

<cdata 'char[]' buffer len 16 from 'bytes' object>

In [179]:
ffi.new('unsigned char[]',ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:])

<cdata 'unsigned char[]' owning 17 bytes>

In [215]:
ffi.new('unsigned char[]',16)

<cdata 'unsigned char[]' owning 16 bytes>

In [211]:
ffi.buffer(grantedAuth.bootstrap_config,int(grantedAuth.bootstrap_config_len))[:]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

In [ ]:
ffi.sizeof()

In [67]:
grantedAuth.bootstrap_config_cap

16

In [209]:
ffi.unpack(grantedAuth.bootstrap_config,16)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [157]:
grantedAuth

<cdata 'AuthGranted *' owning 336 bytes>

In [159]:
ffi.

(['AccessContInfo',
  'AccessContainerEntry',
  'AccountInfo',
  'App',
  'AppAccess',
  'AppExchangeInfo',
  'AppKeys',
  'AsymNonce',
  'AsymPublicKey',
  'AsymSecretKey',
  'AuthGranted',
  'AuthReq',
  'CipherOptHandle',
  'ContainerInfo',
  'ContainerPermissions',
  'ContainersReq',
  'EncryptPubKeyHandle',
  'EncryptSecKeyHandle',
  'FfiResult',
  'File',
  'FileContextHandle',
  'MDataAction',
  'MDataEntriesHandle',
  'MDataEntry',
  'MDataEntryActionsHandle',
  'MDataInfo',
  'MDataKey',
  'MDataPermissionsHandle',
  'MDataValue',
  'MetadataResponse',
  'ObjectHandle',
  'PermissionSet',
  'SEReaderHandle',
  'SEWriterHandle',
  'SelfEncryptorReaderHandle',
  'SelfEncryptorWriterHandle',
  'ShareMData',
  'ShareMDataReq',
  'SignPubKeyHandle',
  'SignPublicKey',
  'SignSecKeyHandle',
  'SignSecretKey',
  'SymNonce',
  'SymSecretKey',
  'UserPermissionSet',
  'XorNameArray',
  'int16_t',
  'int32_t',
  'int64_t',
  'int8_t',
  'int_fast16_t',
  'int_fast32_t',
  'int_fast64_t'

In [158]:
grantedAuth

SyntaxError: invalid syntax (<ipython-input-158-8cc55b270ca7>, line 1)

In [94]:
lib.app_registered(myAuth.app.name,testDataAuth,ffi.NULL,disconnect_app_registered,result_app_registered)

results
140247862083583
b'Serialisation error'


In [148]:
def printCrustConfig(configAddress,configlength=42):
    lali=[]
    for i in range(configlength):
        lali.append(configAddress[i])
    print(lali)

In [149]:
printCrustConfig(testDataAuth.bootstrap_config)

[0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 1, 0, 1, 0]


In [136]:
lali=[]
for i in range(16):
    lali.append(testDataAuth.bootstrap_config[i])
print(lali)

[0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 1, 0, 1, 0]


In [71]:
testDataAuth.bootstrap_config_len

16

In [134]:
testDataAuth=ffi.new('AuthGranted *',grantedAuth[0])

In [93]:
ffi.string(myAuth.app.id)

b'ridsTest'

In [63]:
grantedAuth.access_container_info.id

<cdata 'unsigned char[32]' 0x7ffd1d44c6e0>

In [64]:
type(grantedAuth.access_container_info.tag)

int

In [65]:
grantedAuth.access_container_info

<cdata 'AccessContInfo &' 0x7ffd1d44c6e0>

In [66]:
grantedAuth.access_container_info.tag

15000

In [67]:
grantedAuth.access_container_entry.containers_len

0

In [68]:
grantedAuth.access_container_info.nonce

<cdata 'unsigned char[24]' 0x7ffd1d44c708>

In [69]:
grantedAuth.bootstrap_config_len

16

In [70]:
grantedAuth.bootstrap_config_cap

16

---

In [65]:
grantedAuth

<cdata 'AuthGranted *' owning 336 bytes>

In [66]:
grantedAuth.access_container_info.id

<cdata 'unsigned char[32]' 0x560214c901e0>

In [67]:
type(grantedAuth.access_container_info.tag)

int

In [68]:
grantedAuth.access_container_info

<cdata 'AccessContInfo &' 0x560214c901e0>

In [69]:
grantedAuth.access_container_info.tag

15000

In [70]:
grantedAuth.access_container_entry.containers_len

0

In [71]:
grantedAuth.access_container_info.nonce

<cdata 'unsigned char[24]' 0x560214c90208>

In [72]:
grantedAuth.bootstrap_config_len

16

In [73]:
grantedAuth.bootstrap_config_cap

16

## looking into the granted authentication

In [58]:
def printArray(array):
    arr=[]
    for element in array:
        arr.append(element)
    print(arr)

In [59]:
grantedAuth.app_keys.owner_key

<cdata 'unsigned char[32]' 0x7ffea2566a90>

In [56]:
len(grantedAuth.app_keys.owner_key)

32

In [59]:
printArray(grantedAuth.app_keys.owner_key)

[176, 107, 43, 207, 210, 127, 0, 0, 8, 235, 13, 192, 210, 127, 0, 0, 176, 107, 43, 207, 210, 127, 0, 0, 48, 238, 38, 207, 210, 127, 0, 0]


In [62]:
len(grantedAuth.app_keys.enc_key)

32

In [61]:
len(grantedAuth.app_keys.sign_pk)

32

In [62]:
len(grantedAuth.app_keys.sign_sk)

64

In [63]:
len(grantedAuth.app_keys.enc_pk)

32

In [64]:
len(grantedAuth.app_keys.enc_sk)

32